# Introduction

Mnist digit classification is a project involving image recognition wich we have to cassify images of handwritten digits as 0 to 9.

# Import required modules

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file 

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

from pylab import plt
plt.style.use ('seaborn')
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

keras.backend.set_image_data_format('channels_last')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.utils import plot_model

# Read Directories & Folders

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load the Data

In [ ]:
mnist_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
mnist_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

display(" train data",mnist_train )
display(" test data",mnist_test )

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [ ]:
# Convert to trian and test data; Preserve original dataset
X_train = mnist_train.drop('label', axis=1).copy()
X_test = mnist_test.copy()
Y_train = mnist_train['label'].copy()

In [ ]:
print (f'The train dataset has shape {X_train.shape} and the labels shape is: {Y_train.shape}')
print (f'The test dataset has shape {X_test.shape}')

In [ ]:
# Reshape to 28 x 28 so that we can see the image ie. handwritten number
# resahping the train inputs to be 4D arrays so we can be able to load them
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

In [ ]:
# plot some of the data with their labels as titles:
fig, ax= plt.subplots (2, 8, figsize= [12,4])
for i in range (2):
    for j in range (8):
        rnd= np.random.randint (len(X_train))
        ax[i, j].imshow (X_train[rnd])
        ax[i, j].set_title (Y_train [rnd])
        ax[i, j].set_xticklabels([])
        ax[i, j].set_yticklabels([])

In [ ]:
# Some preprocess on data:

X_train= X_train.reshape ((X_train.shape[0], -1))  ### flatten the train inputs to be 1D arrays
X_test= X_test.reshape ((X_test.shape[0], -1))     ### flatten the test inputs to be 1D arrays

print (X_train.shape, X_test.shape)

Y_train= to_categorical(Y_train, num_classes=10, dtype="int")   ###  One-hot encode the labels to make them real classes, due to do classification

We will normalize the input dataset. This is generally a good idea to avoide some issues like "nan loss"

In [ ]:
X_train= X_train / 255
X_test= X_test / 255

# A Model

Here a simple model with a few hidden layers is used.
In new updates, I will add methods on how to prevent overfitting and also strengthen the model...

In [ ]:
model= Sequential ()

# INputs and first HIDDEN layer
model.add (Dense (512, input_shape= (X_train.shape[1], )))
model.add (Activation ('relu'))

# HIDDEN layers
model.add (Dense (256))
model.add (Dense (128))

# OUTput
model.add (Dense (10))
model.add (Activation('softmax'))

### Visualize the model's detail
print (model.summary())
plot_model (model, show_shapes=True)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history= model.fit (X_train, Y_train, epochs= 50, batch_size= 32, validation_split= 0.3, verbose= 1, callbacks= [])

To see how our model is going, we plot "Learning Curves"

In [ ]:
plt.plot(history.history ['loss'], label= 'loss')
plt.plot(history.history ['val_loss'], label= 'val_loss')

plt.legend ()

In [ ]:
plt.plot(history.history ['accuracy'], label= 'accuracy')
plt.plot(history.history ['val_accuracy'], label= 'val_accuracy')

plt.legend ()

In [ ]:
results = model.predict(X_test)

# select the indices with the maximum probability
results = np.argmax(results,axis = 1) 
results = pd.Series(results,name="Label")
results